# instalación de paquetes y verificación

In [48]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
import string
from datetime import datetime
python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.6.9  which is compatible with this notebook


In [2]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 models/research/object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/test/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feat

# Variables de entorno

In [49]:
training_dir = "training_masks_selected"
images_dir  = "images_mask_selected"
inference_graph_dir = "inference_graph_mask_rcnn_{}_{}_MVI_images".format(datetime.now().day,datetime.now().month)

# creación del dataset

In [9]:
# borramos las carpetas de train test y validación si es que existían de un entrenamiento anterior
rm -r {images_dir}/train {images_dir}/test {images_dir}/val

rm: cannot remove 'images_mask_selected/train': No such file or directory


In [10]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test.
# el argumento -f indica el formato de las imágenes
!python3 scripts/train_test_split.py -i {images_dir} -o {images_dir} -tr 0.1 -vr 0.15 -f jpg
!tree -d {images_dir}

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images_mask_selected
├── test
├── train
└── val

3 directories


In [11]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py {images_dir}/train {images_dir}/train/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/test {images_dir}/test/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/val {images_dir}/val/coco --labels {training_dir}/labels.txt
!tree -d {images_dir}

Creating dataset: images_mask_selected/train/coco
Generating dataset from: images_mask_selected/train/MVI_0100_frame12500.json
Generating dataset from: images_mask_selected/train/MVI_0076_frame2900.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame1800.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame3200.json
Generating dataset from: images_mask_selected/train/MVI_0084_frame7800.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame4900.json
Generating dataset from: images_mask_selected/train/MVI_0098_frame6920.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame4400.json
Generating dataset from: images_mask_selected/train/MVI_0084_frame10360.json
Generating dataset from: images_mask_selected/train/MVI_0084_frame7600.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame8600.json
Generating dataset from: images_mask_selected/train/MVI_0084_frame7960.json
Generating dataset from: images_mask

Generating dataset from: images_mask_selected/train/MVI_0098_frame6840.json
Generating dataset from: images_mask_selected/train/MVI_0076_frame3200.json
Generating dataset from: images_mask_selected/train/MVI_0084_frame10800.json
Generating dataset from: images_mask_selected/train/MVI_0076_frame11200.json
Generating dataset from: images_mask_selected/train/MVI_0100_frame2600.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame7500.json
Generating dataset from: images_mask_selected/train/MVI_0100_frame12600.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame10100.json
Generating dataset from: images_mask_selected/train/MVI_0076_frame12100.json
Generating dataset from: images_mask_selected/train/MVI_0076_frame3800.json
Generating dataset from: images_mask_selected/train/MVI_0076_frame4000.json
Generating dataset from: images_mask_selected/train/MVI_0076_frame3700.json
Generating dataset from: images_mask_selected/train/MVI_0074_frame3100.json
Generat

Generating dataset from: images_mask_selected/val/MVI_0076_frame12000.json
Generating dataset from: images_mask_selected/val/MVI_0076_frame5200.json
Generating dataset from: images_mask_selected/val/MVI_0074_frame6700.json
Generating dataset from: images_mask_selected/val/MVI_0076_frame6000.json
Generating dataset from: images_mask_selected/val/MVI_0084_frame10560.json
Generating dataset from: images_mask_selected/val/MVI_0074_frame7900.json
Generating dataset from: images_mask_selected/val/MVI_0076_frame5400.json
Generating dataset from: images_mask_selected/val/MVI_0098_frame6880.json
Generating dataset from: images_mask_selected/val/MVI_0076_frame11400.json
Generating dataset from: images_mask_selected/val/MVI_0076_frame3400.json
Generating dataset from: images_mask_selected/val/MVI_0084_frame2000.json
Generating dataset from: images_mask_selected/val/MVI_0074_frame6000.json
Generating dataset from: images_mask_selected/val/MVI_0074_frame7800.json
Generating dataset from: images_mas

<img src="labels_txt.PNG">

In [13]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm {images_dir}/test/*
!mv {images_dir}/test/coco/* {images_dir}/test
!rm -rf {images_dir}/test/coco 

!rm {images_dir}/train/*
!mv {images_dir}/train/coco/* {images_dir}/train
!rm -rf {images_dir}/train/coco 

!rm {images_dir}/val/*
!mv {images_dir}/val/coco/* {images_dir}/val
!rm -rf {images_dir}/val/coco 

!tree -d {images_dir}

rm: cannot remove 'images_mask_selected/test/coco': Is a directory
rm: cannot remove 'images_mask_selected/train/coco': Is a directory
rm: cannot remove 'images_mask_selected/val/coco': Is a directory
images_mask_selected
├── test
│   ├── JPEGImages
│   └── Visualization
├── train
│   ├── JPEGImages
│   └── Visualization
└── val
    ├── JPEGImages
    └── Visualization

9 directories


In [14]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 models/research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir={images_dir}/train \
--val_image_dir={images_dir}/val \
--test_image_dir={images_dir}/test \
--train_annotations_file={images_dir}/train/annotations.json \
--val_annotations_file={images_dir}/val/annotations.json \
--testdev_annotations_file={images_dir}/test/annotations.json \
--output_dir={training_dir}/tfrecord \
--include_masks=True

I1029 08:53:16.149774 140521798604608 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I1029 08:53:16.150257 140521798604608 create_coco_tf_record.py:412] 0 images are missing annotations.
I1029 08:53:16.150467 140521798604608 create_coco_tf_record.py:441] On image 0 of 176
I1029 08:53:34.249097 140521798604608 create_coco_tf_record.py:441] On image 100 of 176
I1029 08:53:48.184455 140521798604608 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I1029 08:53:48.196029 140521798604608 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I1029 08:53:48.196166 140521798604608 create_coco_tf_record.py:412] 0 images are missing annotations.
I1029 08:53:48.196207 140521798604608 create_coco_tf_record.py:441] On image 0 of 35
I1029 08:53:54.918287 140521798604608 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I1029 08:53:54.923122 140521798604608 create_coco_tf_record.py:39

# Entrenamiento y exportación del modelo

In [46]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training_masks
!python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir={training_dir}/ \
--pipeline_config_path={training_dir}/mask_rcnn_inception_v2_coco.config \
--num_train_steps=400000

W1029 17:05:28.658907 140458912401216 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 400000
I1029 17:05:28.659050 140458912401216 config_util.py:552] Maybe overwriting train_steps: 400000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1029 17:05:28.659099 140458912401216 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1029 17:05:28.659136 140458912401216 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1029 17:05:28.659174 140458912401216 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1029 17:05:28.659224 140458912401216 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_

Instructions for updating:
Use `tf.cast` instead.
W1029 17:05:40.670685 140458912401216 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I1029 17:05:43.226755 140458912401216 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__` method instead.
W1029 17:05:43.251860 140458912401216 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1029 17:05:44.188454 140458912401216 regularizers.py:99] Scale of 0 di

INFO:tensorflow:Running local_init_op.
I1029 17:05:53.824638 140458912401216 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I1029 17:05:54.056279 140458912401216 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into training_masks_selected/model.ckpt.
I1029 17:05:58.846395 140458912401216 basic_session_run_hooks.py:606] Saving checkpoints for 0 into training_masks_selected/model.ckpt.
2020-10-29 17:06:04.273584: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-10-29 17:06:06.460655: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-10-29 17:06:10.365692: W tensorflow/stream_executor/cuda/ptxas_utils.cc:116] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect res

# para entrenarlo en la terminal copiar y pegar este fragmento

training_dir=training_masks_selected
python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir=$training_dir/ \
--pipeline_config_path=$training_dir/mask_rcnn_inception_v2_coco.config \
--num_train_steps=400000

In [50]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir {training_dir}

checkpoint
eval_0
events.out.tfevents.1603991149.bigdata-alineware
events.out.tfevents.1603991202.bigdata-alineware
events.out.tfevents.1603991247.bigdata-alineware
events.out.tfevents.1604050172.bigdata-alineware
events.out.tfevents.1604052895.bigdata-alineware
events.out.tfevents.1604068772.bigdata-alineware
graph.pbtxt
label_map.pbtxt
labels.txt
mask_rcnn_inception_v2_coco.config
model.ckpt-48666.data-00000-of-00001
model.ckpt-48666.index
model.ckpt-48666.meta
model.ckpt-48667.data-00000-of-00001
model.ckpt-48667.index
model.ckpt-48667.meta
model.ckpt-48668.data-00000-of-00001
model.ckpt-48668.index
model.ckpt-48668.meta
model.ckpt-48669.data-00000-of-00001
model.ckpt-48669.index
model.ckpt-48669.meta
model.ckpt-48670.data-00000-of-00001
model.ckpt-48670.index
model.ckpt-48670.meta
tfrecord


In [51]:
# por si no lo ves claro, corriendo esto te dirá qué debes poner en el script siguiente que generará el grafo de inferencia
import glob
import os
max_step=0
for file in glob.glob(training_dir+"/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
        model_checkpoint_prefix="model.ckpt-{}".format(max_step)
print("En el campo trained_checkpoint debes poner lo siguiente: \n{}/model.ckpt-".format(training_dir)+str(max_step))

En el campo trained_checkpoint debes poner lo siguiente: 
training_masks_selected/model.ckpt-48670


In [55]:
# borramos el grafo de inferencia en caso de que queramos
!rm -r {inference_graph_dir}

In [52]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido del modelo entrenado y la fecha y la hora
!python3 models/research/object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path {training_dir}/mask_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix {training_dir}/{model_checkpoint_prefix} \
--output_directory {inference_graph_dir}

Instructions for updating:
Please use `layer.__call__` method instead.
W1103 08:14:00.560460 140209898223424 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I1103 08:14:01.412139 140209898223424 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I1103 08:14:01.536967 140209898223424 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1103 08:14:01.537248 140209898223424 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1103 08:14:01.576214 140

244 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*,.*Initializer.*,.*Regularizer.*,.*BiasAdd.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organiz

2020-11-03 08:14:04.700440: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-03 08:14:04.850513: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:17:00.0
2020-11-03 08:14:04.851768: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: GeForce GTX 1080 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:65:00.0
2020-11-03 08:14:04.895886: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-11-03 08:14:04.941373: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-11-03 08:14:04.996809: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W1103 08:14:16.311208 140209898223424 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/tools/freeze_graph.py:233: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
W1103 08:14:16.311568 140209898223424 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/framework/graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 360 variables.
I1103 08:14:16.750217

In [53]:
!python3 scripts/tf_text_graph_mask_rcnn.py \
--input {inference_graph_dir}/frozen_inference_graph.pb \
--config {training_dir}/mask_rcnn_inception_v2_coco.config \
--output {inference_graph_dir}/graph.pbtxt

Number of classes: 5
Scales:            [0.25, 0.5, 1.0, 2.0]
Aspect ratios:     [0.5, 1.0, 2.0]
Width stride:      16.000000
Height stride:     16.000000
Features stride:   16.000000


# Generando predicciones en imágenes

In [54]:
!python3 scripts/predict_on_images.py \
-l via catenaria pk senal_fija senal_luminosa \
-m {inference_graph_dir} \
-i unlabeled \
-o predictions_masks_03_11_MVI_images \
-em 1 2 3 4 \
-eb 0 \
-t 0.6 \
-f jpg

output directory doesn't exist, creating it
predictions_masks_03_11_MVI_images/MVI_0098_frame6360_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0084_frame10960_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0084_frame9880_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0098_frame7360_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame3480_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0101_frame12480_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame3720_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame7360_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100_frame3520_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0076_frame3760_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0084_frame5000_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0076_frame11520_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0098_frame6440_predictions.jpg
predictions_masks_03_11_MVI_images/MVI_0100

KeyboardInterrupt: 